# Building a multi-user ReAct AI Agent Chatbot with LangGraph - Prebuilt Agents


1) Create tools and Test with LLM

2) Build and Test AI Agent with LangGraph Pre-built Agent

3) Build and Test Multi-User Conversational ReAct AI Agent

In [6]:
!pip install langchain
!pip install langchain_openai
!pip install langchain_community
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is

In [1]:
from getpass import getpass
OPENAI_API_KEY=getpass("enter open ai key :-")

enter open ai key :-··········


In [2]:
TAVILY_API_KEY=getpass("enter TAVILY_API_KEY :-")

enter TAVILY_API_KEY :-··········


In [3]:
WEATHER_API_KEY=getpass("enter WEATHER_API_KEY:-")

enter WEATHER_API_KEY:-··········


In [4]:
import os
os.environ["WEATHER_API_KEY"]=WEATHER_API_KEY
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [7]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o",temperature=0)

#1)Create tools and Test with LLM

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import requests
import json

web_search=TavilySearchResults(max_results=4,max_tokens=10000,search_depth='advanced')

@tool
def found_on_web(query:str)->list:
  """a tool for search the new , event or any information on web related to current event or past or anything."""
  response=web_search.invoke(query)
  return response

@tool
def fetch_weather(query:str)->list:
  """a tool for fetching weather information."""
  base_url="http://api.weatherapi.com/v1/current.json"
  complete_url=f"{base_url}?key={WEATHER_API_KEY}&q={query}"
  response=requests.get(complete_url)
  data=response.json()
  if data.get("location"):
    return data
  else:
    return "Weather data not found."

/tmp/ipython-input-2093383909.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search=TavilySearchResults(max_results=4,max_tokens=10000,search_depth='advanced')


__Testing of tools__

In [9]:
llm_tool=llm.bind_tools(tools=[found_on_web,fetch_weather])

In [13]:
llm_tool.invoke("ajmer weather")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kCGvrg10Shet2LI0Yl6x17e4', 'function': {'arguments': '{"query":"Ajmer"}', 'name': 'fetch_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CGj0QPbu1RgH1dgfaMlK1naTs7dv8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4c362e41-cb23-4c01-9e0d-8694212e9955-0', tool_calls=[{'name': 'fetch_weather', 'args': {'query': 'Ajmer'}, 'id': 'call_kCGvrg10Shet2LI0Yl6x17e4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 15, 'total_tokens': 99, 'input_token_details': {'audio

#2) Build and Test AI Agent with LangGraph Pre-built Agent

In [10]:
#we need to prepare the prompt who direct the llm in agent for what need to do and how to do ?

from langchain_core.prompts import ChatPromptTemplate

sys_prompt="""You are an intelligent **AI Assistant**.
You have access to tools, each with specific functionality:
1) found_on_web :-a tool for search the new , event or any information on web related to current event or past or anything.
2) fetch_weather :- a tool for fetching weather information.

### 🔑 Rules:
1. **Always** select the correct tool that matches the user query.
2. If the query requires multiple steps, break it down and call the tools in the right sequence.
3. Do not invent or assume tools outside the provided tools.
4. If none of the tools apply, respond politely that you cannot complete the request.
5. Always keep responses concise, clear, and aligned with the tool outputs.

### 🎯 Objective:
Your primary role is to **help the user by reasoning about their request and calling the appropriate tools**.
After getting the tool output, integrate it into a **final, helpful answer** for the user.
"""

In [11]:
from langchain_core.prompts import MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
    ("system",sys_prompt),
    MessagesPlaceholder(variable_name="messages")
])

In [19]:
from langgraph.prebuilt import create_react_agent
agent=create_react_agent(model=llm,tools=[found_on_web,fetch_weather],prompt=prompt)

In [20]:
response=agent.invoke({"messages":[("user","weather in ajmer")]})

In [33]:
print(response['messages'][-1].content)

The current weather in Ajmer, Rajasthan, India is sunny with a temperature of 32.7°C (90.9°F). The wind is blowing from the northwest at 11.9 kph (7.4 mph). The humidity level is at 45%, and there is no precipitation. The weather feels like 35.0°C (95.1°F) due to the heat index. Visibility is good at 10 km (6 miles), and the UV index is 3.5.


In [34]:
response=agent.invoke({"messages":[("user","how much current tariff on india by usa ?why so ?")]})
print(response['messages'][-1].content)

The current tariffs imposed by the USA on India are primarily due to India's continued purchases of Russian oil and weapons. The tariffs, which have been increased to 50%, include a 25% penalty specifically for transactions with Russia. The US government, under the Trump administration, implemented these tariffs as a measure to pressure India to reduce its economic ties with Russia, which the US views as supporting Russia's actions in Ukraine. These tariffs are among the highest in the world and have been criticized by India as unfair, with India emphasizing its need to secure the best deals for its energy needs to support its large population.


In [35]:
response=agent.invoke({"messages":[("user","is it justified ?")]})
print(response['messages'][-1].content)

Could you please provide more context or specify what you are referring to? This will help me understand your question better and provide a more accurate response.


__Because the agent unable to read the history of conversation these all things happens.__

#3) Build and Test Multi-User Conversational ReAct AI Agent

In [13]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
memory=MemorySaver()
agent=create_react_agent(model=llm,checkpointer=memory,prompt=prompt,tools=[found_on_web,fetch_weather])

In [16]:
response=agent.invoke({"messages":[("user","weather in ajmer")]},{"configurable":{"thread_id":"gld123"}})
print(response['messages'][-1].content)

The current weather in Ajmer, Rajasthan, India is sunny with a temperature of 32.6°C (90.7°F). The wind is blowing from the west at 14.8 kph (9.2 mph), and the humidity is at 44%. The weather feels like 34.6°C (94.2°F). There is no precipitation, and the visibility is 10 km. The UV index is 9.2, indicating a high level of UV radiation.


In [17]:
response=agent.invoke({"messages":[("user","how much current tariff on india by usa ?why so ?")]},{"configurable":{"thread_id":"gld124"}})
print(response['messages'][-1].content)

The current tariffs imposed by the USA on India are quite significant, with some goods facing tariffs as high as 50%. These tariffs were initially imposed by the Trump administration as a response to India's purchases of Russian oil and weapons, which the US viewed as supporting Russia's actions in Ukraine. The tariffs are among the highest globally and have affected various sectors, including textiles, shrimp, diamonds, and auto components, which are labor-intensive and crucial for India's economy.

The tariffs have been criticized as unfair by the Indian government, and they have led to a decrease in India's exports to the US, affecting the price competitiveness of Indian goods. The geopolitical tensions, particularly India's alignment with Russia and China, have further complicated the situation, impacting US-India relations.


In [18]:
response=agent.invoke({"messages":[("user","is it justified ?")]},{"configurable":{"thread_id":"gld124"}})
print(response['messages'][-1].content)

The justification for tariffs is often subjective and depends on the perspective from which you view the situation. Here are some points to consider:

1. **From the US Perspective**: The tariffs were imposed as a strategic move to pressure India to reduce its economic ties with Russia, particularly in the context of Russia's actions in Ukraine. The US might view this as a necessary step to uphold its foreign policy objectives and to discourage support for Russia.

2. **From the Indian Perspective**: India considers these tariffs as unfair and unjustified, especially since they impact critical sectors of its economy. India argues that it has the right to pursue its own economic interests, including purchasing oil and weapons from Russia, to ensure energy security and defense needs.

3. **Economic Impact**: The tariffs have economic consequences for both countries. They can lead to increased costs for US consumers and businesses that rely on Indian imports, and they can harm Indian expor

In [19]:
response=agent.invoke({"messages":[("user","is it justified ?")]},{"configurable":{"thread_id":"gld123"}})
print(response['messages'][-1].content)

Could you please clarify what you mean by "is it justified?" Are you asking if the weather conditions are typical for Ajmer at this time of year, or are you referring to something else?


__Note:- we can see easily in above session , the magic of memory through checkpointer.now conversation history can saved.__